In [ ]:
## Global Requirements :

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from osgeo import gdal
import sys
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data collection :

## 1- Field Measurements :

In [ ]:
## Install Requirements : 

!pip install cartopy

     |████████████████████████████████| 12.1 MB 151 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cartopy: filename=Cartopy-0.19.0.post1-cp37-cp37m-linux_x86_64.whl size=12516271 sha256=8ba6fb160e9220f2d350ead86b8dea8a913af08397638c3455b35dd129830455
  Stored in directory: /root/.cache/pip/wheels/98/01/f7/bd10aeb96fe4b518cde5f7c4f5e12c7202f85b7353a5017847
Successfully built cartopy


In [ ]:
## Install Requirements : 
!pip install ismn  

     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 212 kB 10.4 MB/s 
     |████████████████████████████████| 6.6 MB 58.3 MB/s 


In [ ]:
from ismn.interface import ISMN_Interface

In [ ]:
## Process tha data file with the ISMN interface : 

data_path = "/content/drive/MyDrive/Datasets/Data_separate_files_header_20170301_20201231_8964_imyy_20210812"
ismn_data = ISMN_Interface(data_path, parallel=True)

Files Processed:   0%|          | 0/1 [00:00<?, ?it/s]

Processing metadata for all ismn stations into folder /content/drive/MyDrive/Datasets/Data_separate_files_header_20170301_20201231_8964_imyy_20210812.
This may take a few minutes, but is only done once...



Files Processed: 100%|██████████| 1/1 [00:01<00:00,  1.63s/it]

Metadata generation finished after 1 Seconds.
Metadata and Log stored in /content/drive/MyDrive/Datasets/Data_separate_files_header_20170301_20201231_8964_imyy_20210812/python_metadata
Found existing ismn metadata in /content/drive/MyDrive/Datasets/Data_separate_files_header_20170301_20201231_8964_imyy_20210812/python_metadata/Data_separate_files_header_20170301_20201231_8964_imyy_20210812.csv.


In [ ]:
ismn_data.collection

REMEDHUS: ['Casa_Periles']

In [ ]:
 ismn_data.collection['REMEDHUS']['Casa_Periles'].metadata

MetaData([
  MetaVar([clay_fraction, 21.0, Depth([0.0, 0.3])]),
  MetaVar([climate_KG, BSk, None]),
  MetaVar([climate_insitu, unknown, None]),
  MetaVar([elevation, -99.9, None]),
  MetaVar([instrument, Stevens-Hydra-Probe, Depth([0.0, 0.05])]),
  MetaVar([latitude, 41.39392, None]),
  MetaVar([lc_2000, 10, None]),
  MetaVar([lc_2005, 10, None]),
  MetaVar([lc_2010, 10, None]),
  MetaVar([lc_insitu, unknown, None]),
  MetaVar([longitude, -5.32146, None]),
  MetaVar([network, REMEDHUS, None]),
  MetaVar([organic_carbon, 0.65, Depth([0.0, 0.3])]),
  MetaVar([sand_fraction, 36.0, Depth([0.0, 0.3])]),
  MetaVar([saturation, 0.43, Depth([0.0, 0.3])]),
  MetaVar([silt_fraction, 43.0, Depth([0.0, 0.3])]),
  MetaVar([station, Casa_Periles, None]),
  MetaVar([timerange_from, 2017-03-01 00:00:00, None]),
  MetaVar([timerange_to, 2020-12-31 23:00:00, None]),
  MetaVar([variable, soil_moisture, Depth([0.0, 0.05])]),
  MetaVar([variable, soil_temperature, Depth([0.0, 0.05])])
])

In [ ]:
 ismn_data.collection['REMEDHUS']['Casa_Periles']

Sensors at 'Casa_Periles': ['Stevens-Hydra-Probe_soil_moisture_0.000000_0.050000', 'Stevens-Hydra-Probe_soil_temperature_0.000000_0.050000']

In [ ]:
L=['Stevens-Hydra-Probe_soil_moisture_0.000000_0.050000']

In [ ]:
## Utilities :

def get_sensor_by(network, station):
  df = []
  
  for index,value in enumerate(ismn_data.collection[network][station]) :
    #print("soil_moisture" in str(value),index)
    #print("--------------")
    sensor = ismn_data.collection[str(network)][str(station)][str(value)]
    
    if str(value) in L :

      ts = sensor.read_data()
      ts.drop('soil_moisture_orig_flag', inplace=True, axis=1)  
      ts.drop('soil_moisture_flag', inplace=True, axis=1)  
      ts["depth"] = float(str(value).split("_")[-1])
      ts.rename(columns={'soil_moisture': 'soil_moisture_depth_'+str(value).split("_")[-1]},inplace=True)
      df.append(ts)
      allS = pd.concat(df)

  
  return allS

def get_station_by(network):
  df1 = []
  for index,key in enumerate(ismn_data.collection[network]) :
    #print(get_name(key))
    data = get_sensor_by(network, get_name(key))
    data["station"] = get_name(key)
    df1.append(data)

  allN = pd.concat(df1)
  allN.head()
  return allN 



In [ ]:
## Get The dataset of the network-station desired : 

dataset = get_sensor_by('REMEDHUS','Casa_Periles')

In [ ]:
pd.set_option("display.max_rows", False)

In [ ]:
dataset.head()

,soil_moisture_depth_0.050000,depth
date_time,,
2017-03-01 00:00:00,0.156,0.05
2017-03-01 01:00:00,0.156,0.05
2017-03-01 02:00:00,0.156,0.05
2017-03-01 03:00:00,0.155,0.05
2017-03-01 04:00:00,0.155,0.05


In [ ]:
dataset.describe()

,soil_moisture_depth_0.050000,depth
count,32922.000000,3.292200e+04
mean,0.156274,5.000000e-02
std,0.057528,2.883848e-14
min,0.053000,5.000000e-02
25%,0.109000,5.000000e-02
50%,0.153000,5.000000e-02
75%,0.202000,5.000000e-02
max,0.371000,5.000000e-02


In [ ]:
## Save the dataset as csv file :

dataset.to_csv ('SMC_dataset_2017to2020_Area3.csv', header=True)

In [ ]:
## Check if the file is correctly saved :

df = pd.read_csv('SMC_dataset_2017to2020_Area3.csv')
df.tail()

,date_time,soil_moisture_depth_0.050000,depth
33193,2020-12-31 19:00:00,0.181,0.05
33194,2020-12-31 20:00:00,0.183,0.05
33195,2020-12-31 21:00:00,0.185,0.05
33196,2020-12-31 22:00:00,0.186,0.05
33197,2020-12-31 23:00:00,0.186,0.05


## 2- Satellite Imagery :

In [ ]:
## Requirements : 

import ee
import folium
from IPython.display import Image
from datetime import datetime
from datetime import timedelta

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=WFhJY1l0g_Z4y8aOuJATadnYxtZ2nsvdrY_NzHPSkLY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh2wyRadbgrXx5KMyIMnqiPtudQeYnYYvISC33JYMu5dcOqAx0VPsM

Successfully saved authorization token.


In [ ]:
# Initialize the library.
ee.Initialize()

### Selection of a the region

In [ ]:
pip install geemap

     |████████████████████████████████| 476 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 35.4 MB/s 
     |████████████████████████████████| 1.6 MB 34.0 MB/s 
     |████████████████████████████████| 130 kB 58.5 MB/s 
     |████████████████████████████████| 3.3 MB 43.7 MB/s 
     |████████████████████████████████| 1.3 MB 64.4 MB/s 
     |████████████████████████████████| 76 kB 5.5 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 219 kB 59.2 MB/s 
     |████████████████████████████████| 98 kB 5.4 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 115 kB 58.8 MB/s 
     |████████████████████████████████| 551 kB 73.7 MB/s 
     |████████████████████████████████| 392 kB 61.8 MB/s 
     |████████████████████████████████| 112 kB 58.7 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
     |█████████████████████

In [ ]:
import geemap

In [ ]:
## Downloading images of the region :

lat1 =  -5.323319733758855
long1 = 41.40575983571157
lat2 =  -5.3226424171725295
long2 = 41.39042513007489 
lat3 = -5.34185 
long3 = 41.39001 
lat4 =  -5.342346172411098
long4 = 41.404466686988584


aoi = ee.Geometry.Polygon([[[lat1, long1], [lat2, long2], [lat3, long3], [lat4, long4]]])

CLOUD_FILTER = 60

time_range=['2017-03-29', '2020-12-31']

sentinel_filtered = (ee.ImageCollection('COPERNICUS/S2_SR').
                         filterBounds(aoi).
                         select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12']).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
                         )


geemap.ee_export_image_collection_to_drive(sentinel_filtered, folder='Senti_Images-2017to2020_Area3', scale=10, region=aoi)


Total number of images: 642

Exporting 20170412T110621_20170412T111708_T30TUL ...
Exporting 20170422T110651_20170422T111300_T29TQG ...
Exporting 20170502T110621_20170502T110937_T29TQF ...
Exporting 20170502T110621_20170502T110937_T29TQG ...
Exporting 20170502T110621_20170502T110937_T30TTL ...
Exporting 20170502T110621_20170502T110937_T30TUL ...
Exporting 20170512T110651_20170512T111302_T29TQG ...
Exporting 20170512T110651_20170512T111302_T30TTL ...
Exporting 20170522T110621_20170522T110912_T29TQF ...
Exporting 20170522T110621_20170522T110912_T29TQG ...
Exporting 20170522T110621_20170522T110912_T30TTL ...
Exporting 20170522T110621_20170522T110912_T30TUL ...
Exporting 20170601T110651_20170601T111225_T29TQF ...
Exporting 20170601T110651_20170601T111225_T29TQG ...
Exporting 20170601T110651_20170601T111225_T30TTL ...
Exporting 20170601T110651_20170601T111225_T30TUL ...
Exporting 20170611T110621_20170611T111012_T29TQF ...
Exporting 20170611T110621_20170611T111012_T29TQG ...
Exporting 2017061

# 3- Dataset :
Synchronize the field measurement with images dates

In [ ]:
## Requirements :

from osgeo import gdal
import sys
import os

In [ ]:
## Read the data from the 2 stations :

df1 = pd.read_csv("/content/drive/MyDrive/soilMoisture/Datasets/SMC_dataset_2017to2020_Area1.csv")
df2 = pd.read_csv("/content/drive/MyDrive/soilMoisture/Datasets/SMC_dataset_2017to2020_Area2.csv")
df3 = pd.read_csv("/content/drive/MyDrive/soilMoisture/Datasets/SMC_dataset_2017to2020_Area3.csv")

In [ ]:
df3.head() 

,date_time,soil_moisture_depth_0.050000,depth
0,2017-03-01 00:00:00,0.156,0.05
1,2017-03-01 01:00:00,0.156,0.05
2,2017-03-01 02:00:00,0.156,0.05
3,2017-03-01 03:00:00,0.155,0.05
4,2017-03-01 04:00:00,0.155,0.05


In [ ]:
df1.columns 

Index(['date_time', 'soil_moisture_depth_0.050000', 'depth'], dtype='object')

In [ ]:
df1.describe()

,soil_moisture_depth_0.050000,depth
count,33198.000000,3.319800e+04
mean,0.120527,5.000000e-02
std,0.051054,2.897726e-14
min,0.040000,5.000000e-02
25%,0.073000,5.000000e-02
50%,0.115000,5.000000e-02
75%,0.168000,5.000000e-02
max,0.271000,5.000000e-02


In [ ]:
## split dates and hours :

df1['date_time'][1].split(" ") 

['2017-03-01', '01:00:00']

In [ ]:
## Add hours column 

df1["hour"] = ""
df2["hour"] = ""
df3["hour"] = ""

for i in range (len(df1)):
  df1["hour"][i]=df1['date_time'][i].split(" ")[1]
  df1["date_time"][i]=df1['date_time'][i].split(" ")[0]

for i in range (len(df2)):
  df2["hour"][i]=df2['date_time'][i].split(" ")[1]
  df2["date_time"][i]=df2['date_time'][i].split(" ")[0]
  
for i in range (len(df3)):
  df3["hour"][i]=df3['date_time'][i].split(" ")[1]
  df3["date_time"][i]=df3['date_time'][i].split(" ")[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.

In [ ]:
df1.head()

,date_time,soil_moisture_depth_0.050000,depth,hour
0,2017-03-01,0.156,0.05,00:00:00
1,2017-03-01,0.156,0.05,01:00:00
2,2017-03-01,0.156,0.05,02:00:00
3,2017-03-01,0.155,0.05,03:00:00
4,2017-03-01,0.155,0.05,04:00:00


In [ ]:
#Rdataset1 = pd.DataFrame(columns=['date_time', 'soil_moisture_depth_0.05','Image_name'])
Rdataset1 = df1[df1['hour']=='11:00:00']
Rdataset2 = df2[df2['hour']=='11:00:00']
Rdataset3 = df3[df3['hour']=='11:00:00']

In [ ]:
## Selection of the max value on the day for each image :

L_max = []
for e in df1['date_time'].unique() : 
  L_mid = list(df1[df1['date_time']==e]['soil_moisture_depth_0.050000'])
  L_max.append(max(L_mid))
L_date = df1['date_time'].unique()

In [ ]:
Rdataset['date_time'] = L_date
Rdataset['soil_moisture_depth_0.05'] = L_max
Rdataset.describe()

,soil_moisture_depth_0.05
count,1401.000000
mean,0.128318
std,0.051830
min,0.047000
25%,0.081000
50%,0.122000
75%,0.172000
max,0.271000


In [ ]:
Rdataset2.head()

,date_time,soil_moisture_depth_0.050000,depth,hour
11,2017-03-01,0.195,0.05,11:00:00
35,2017-03-02,0.194,0.05,11:00:00
59,2017-03-03,0.192,0.05,11:00:00
83,2017-03-04,0.186,0.05,11:00:00
107,2017-03-05,0.189,0.05,11:00:00


In [ ]:
import plotly.figure_factory as ff

np.random.seed(1)

x1 = Rdataset1['soil_moisture_depth_0.050000']
x2 = Rdataset2['soil_moisture_depth_0.050000']
x3 = Rdataset3['soil_moisture_depth_0.050000']
hist_data = [x1,x2,x3]
group_labels = ['SMC1','SMC2','SMC3'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels)
fig.show()

### As you may notice during the downloading process, we have for the same date 3 different extentions for the same image (QF.tif, UL.tif, TL.tif), but the problem it's not the case for all the images. Soo we. have to pick the the extentions that appear for almost all the images, because with don't want to train our model with different inputs.
### P.S: it might not be the case for you, you can only 1 extention or maybe 2, but just make attention to it.

In [ ]:

directory = '/content/drive/MyDrive/soilMoisture/Datasets/SMC_dataset_2017to2020_Area2.csv'

a, b, c,  d = 0, 0, 0, 0
for filename in os.listdir(directory):
    if filename.endswith("TL.tif"):
      a+=1
    if filename.endswith("UL.tif"):
      b+=1
    if filename.endswith("QF.tif"):
      c+=1
    if filename.endswith("QF.tif"):
      d+=1

In [ ]:
a,b,c, d

(159, 169, 156, 156)

In [ ]:
## Get a list of the dates from image name :

data_list =[]
Vals = []

for filename in os.listdir(directory):
    if filename.endswith(".tif"):

      im_date = filename[0:4]+"-"+filename[4:6]+"-"+filename[6:8]
      data_list.append(im_date)

In [ ]:
## Check unique dates:

myset = set(data_list)

In [ ]:
len(data_list)

642

In [ ]:
len(myset)

192

In [ ]:
## Selection of the only useful days and add images names :


directory1 = '/content/drive/MyDrive/soilMoisture/Senti_Images-2017to2020_Area1'
directory2 = '/content/drive/MyDrive/soilMoisture/Senti_Images-2017to2020_Area2'
directory3 = '/content/drive/MyDrive/soilMoisture/Senti_Images-2017to2020_Area3'

data_list =[]
Vals = []

for filename in os.listdir(directory1):
    if filename.endswith("UL.tif"):

      im_date = filename[0:4]+"-"+filename[4:6]+"-"+filename[6:8]

      if im_date in Rdataset1["date_time"].values : 
          index_date= np.where(Rdataset1["date_time"]==im_date )
          rdata = Rdataset1.iloc[index_date]
          data_list.append(rdata)
          dataset1 = pd.concat(data_list)
          Vals.append(filename)
dataset1['Image_name']= Vals

data_list =[]
Vals = []
for filename in os.listdir(directory2):
    if filename.endswith("UL.tif"):

      im_date = filename[0:4]+"-"+filename[4:6]+"-"+filename[6:8]

      if im_date in Rdataset2["date_time"].values : 
          index_date= np.where(Rdataset2["date_time"]==im_date )
          rdata = Rdataset2.iloc[index_date]
          data_list.append(rdata)
          dataset2 = pd.concat(data_list)
          Vals.append(filename)
dataset2['Image_name']= Vals

data_list =[]
Vals = []
for filename in os.listdir(directory3):
    if filename.endswith("UL.tif"):

      im_date = filename[0:4]+"-"+filename[4:6]+"-"+filename[6:8]

      if im_date in Rdataset3["date_time"].values : 
          index_date= np.where(Rdataset3["date_time"]==im_date )
          rdata = Rdataset3.iloc[index_date]
          data_list.append(rdata)
          dataset3 = pd.concat(data_list)
          Vals.append(filename)

dataset3['Image_name']= Vals
dataset3.head()
      

,date_time,soil_moisture_depth_0.050000,depth,hour,Image_name
1019,2017-04-12,0.082,0.05,11:00:00,20170412T110621_20170412T111708_T30TUL.tif
1499,2017-05-02,0.069,0.05,11:00:00,20170502T110621_20170502T110937_T30TUL.tif
1949,2017-05-22,0.087,0.05,11:00:00,20170522T110621_20170522T110912_T30TUL.tif
2188,2017-06-01,0.091,0.05,11:00:00,20170601T110651_20170601T111225_T30TUL.tif
2428,2017-06-11,0.069,0.05,11:00:00,20170611T110621_20170611T111012_T30TUL.tif


In [ ]:
print(len(dataset3))

168


In [ ]:
## Save dataset :

dataset1.to_csv ('Dataset_2017to2020_Area1.csv', header=True, index=False)
dataset2.to_csv ('Dataset_2017to2020_Area2.csv', header=True, index=False)
dataset3.to_csv ('Dataset_2017to2020_Area3.csv', header=True, index=False)

In [ ]:
df1 = pd.read_csv('/content/Dataset_2017to2020_Area1.csv')
df2 = pd.read_csv('/content/Dataset_2017to2020_Area2.csv')
df3 = pd.read_csv('/content/Dataset_2017to2020_Area3.csv')

In [ ]:
frames = [df1,df2,df3]

In [ ]:
big_df = pd.concat(frames)

In [ ]:
big_df.describe()

,soil_moisture_depth_0.050000,depth
count,503.000000,5.030000e+02
mean,0.146402,5.000000e-02
std,0.067994,4.375855e-16
min,0.042000,5.000000e-02
25%,0.094000,5.000000e-02
50%,0.137000,5.000000e-02
75%,0.191000,5.000000e-02
max,0.488000,5.000000e-02


In [ ]:
import plotly.figure_factory as ff

np.random.seed(1)

x = big_df['soil_moisture_depth_0.050000']
hist_data = [x]
group_labels = ['distplot'] # name of the dataset

fig = ff.create_distplot(hist_data, group_labels)
fig.show()

In [ ]:
## Save dataset :

big_df.to_csv ('Dataset_3_Areas.csv', header=True, index=False)